In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

## Create data

In [ ]:
rng = RandomState(20190401)

In [ ]:
# i freq
# amp_level
inputs =  []

for i_class, freq_amps  in enumerate((((2,6), (8,4)), ((2,4), (8,6)))):
    fft_coefs = np.zeros((300, 33), dtype=np.complex64)
    for i_freq, amp in freq_amps:
        amps = np.abs(rng.randn(300) + amp)
        phases = rng.rand(300) * 2 * np.pi - np.pi
        this_fft_coefs_a = amps * np.exp(phases * 1j)
        fft_coefs[:,i_freq] = this_fft_coefs_a
        
    signals = np.fft.irfft(fft_coefs, norm='ortho')
    inputs.append(np_to_var(signals, dtype=np.float32))

In [ ]:
def th_angle(x,y):
    # yes reverse oder to (y,x)...
    return th.atan2(y, x)


def th_amp(x,y):
    return th.sqrt((x * x) + (y * y))

class AmplitudePhaseFFT(nn.Module):
    def __init__(self):
        super(AmplitudePhaseFFT, self).__init__()

    def forward(self, inputs):
        expanded = th.cat(
            (inputs[:, :1], th.zeros(inputs.shape[0], 1, device=inputs.device),
             inputs[:, 1:], th.zeros(inputs.shape[0], 1, device=inputs.device)),
            dim=1)
        unfolded = expanded.view(expanded.shape[0], expanded.shape[1] // 2, 2)
        
        x = unfolded.transpose(len(unfolded.shape)-1,0)[0].transpose(
            len(unfolded.shape) - 2,0)
        y = unfolded.transpose(len(unfolded.shape)-1,0)[1].transpose(
            len(unfolded.shape) - 2,0)
        
        amps = th_amp(x, y)
        phases = th_angle(x, y)
        
        return th.stack((amps, phases), dim=-1)
    
    def invert(self, out):
        amps = out.transpose(len(out.shape) - 1,0)[0].transpose(
            len(out.shape) - 2,0)
        phases = out.transpose(len(out.shape) - 1,0)[1].transpose(
            len(out.shape) - 2,0)
        x = th.cos(phases)
        y = th.sin(phases)
        x_y = th.stack((x,y), dim=-1)
        x_y = x_y * amps.unsqueeze(-1)
        flattened = x_y.view(x_y.shape[0], -1)
        reduced = th.cat((flattened[:, :1], flattened[:, 2:-1]), dim=1)
        return reduced


In [ ]:
from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet

feature_model = nn.Sequential(RFFT(),
                              AmplitudePhaseFFT())


outs = feature_model(inputs[0])

ins = invert(feature_model, outs)

th.mean(np.abs(ins - inputs[0]))

from ot_exact import ot_euclidean_loss_for_samples

train_inputs = inputs
test_inputs = inputs

In [ ]:
from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(RFFT(),
                              AmplitudePhaseFFT())

from reversible.training import hard_init_std_mean
n_dims = train_inputs[0].shape[1]
n_clusters = len(train_inputs)
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]
# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [th.autograd.Variable(th.zeros(n_dims), requires_grad=True)
                    for _ in range(n_clusters)]

for i_class in range(n_clusters):
    this_outs = feature_model(train_inputs[i_class])
    means_per_cluster[i_class].data = th.mean(this_outs, dim=0).view(-1).data
    stds_per_cluster[i_class].data = th.log(th.std(this_outs, dim=0),).view(-1).data



from copy import deepcopy

optim_dist = th.optim.Adam(
                          [
    {'params': means_per_cluster + stds_per_cluster,
    'lr': 1e-2,
    'weight_decay': 0},], betas=(0,0.9))


In [ ]:

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples

from reversible.gaussian import get_gauss_samples
from reversible.uniform import get_uniform_samples
from reversible.revnet import invert 
import pandas as pd
from gradient_penalty import gradient_penalty
import time


df = pd.DataFrame()
g_loss = np_to_var([np.nan],dtype=np.float32)
g_grad = np.nan
d_loss = np_to_var([np.nan],dtype=np.float32)
d_grad = np.nan
gradient_loss = np_to_var([np.nan],dtype=np.float32)

In [ ]:
def invert_hierarchical(features):
    return invert(feature_model, features)


def get_samples(n_samples, i_class):
    mean = means_per_cluster[i_class]
    std = th.exp(stds_per_cluster[i_class])
    # let's create a mask for the std for now
    samples_amp = get_gauss_samples(n_samples, mean[::2], std[::2], truncate_to=3)
    samples_phase = get_uniform_samples(n_samples, mean.detach()[1::2] * 0, std.detach()[1::2] * 0 + 2* np.pi,)
    samples = th.stack((samples_amp, samples_phase), dim=-1)
    return samples

In [ ]:
n_epochs = 1001
rng = RandomState(349384)
for i_epoch in range(n_epochs):
    start_time = time.time()
    optim_dist.zero_grad()
    for i_class in range(len(train_inputs)):
        this_inputs = train_inputs[i_class]
        n_samples = len(this_inputs) * 5
        samples = get_samples(n_samples, i_class)
        inverted = invert_hierarchical(samples)
        g_loss = ot_euclidean_loss_for_samples(this_inputs.view(this_inputs.shape[0],-1),
                              inverted.view(inverted.shape[0],-1))
        g_loss.backward()
    g_grad = np.mean([th.sum(p.grad **2).item() for p in itertools.chain(feature_model.parameters())])
    dist_grad = np.mean([th.sum(p.grad **2).item() for p in  means_per_cluster + stds_per_cluster])
    optim_dist.step()
    with th.no_grad():
        sample_wd_row = {}
        for setname, setinputs in [('train', train_inputs), ('test', test_inputs)]:
            for i_class in range(len(setinputs)):
                this_inputs = setinputs[i_class]
                n_samples = len(this_inputs)
                samples = get_samples(n_samples, i_class)
                inverted = invert_hierarchical(samples)
                in_np = var_to_np(this_inputs).reshape(len(this_inputs), -1)
                fake_np = var_to_np(inverted).reshape(len(inverted), -1)
                import ot

                dist = np.sqrt(np.sum(np.square(in_np[:,None] - fake_np[None]), axis=2))
                match_matrix = ot.emd([],[], dist)
                cost = np.sum(dist * match_matrix)
                sample_wd_row.update({
                    setname + '_sampled_wd' + str(i_class): cost,
                })
        end_time = time.time()
        epoch_row = {
        'g_loss': g_loss.item(),
        'g_grad': g_grad,
        'dist_grad': dist_grad,
        'runtime': end_time -start_time,}
        epoch_row.update(sample_wd_row)
        df = df.append(epoch_row, ignore_index=True)
        if i_epoch % (max(1,n_epochs // 20)) == 0:
            display_text("Epoch {:d}".format(i_epoch))
            display(df.iloc[-5:])
        if i_epoch % (n_epochs // 20) == 0:
            print("stds\n", var_to_np(th.exp(th.stack(stds_per_cluster))))
            fig = plt.figure(figsize=(8,4))
            plt.plot(var_to_np(th.exp(th.stack(stds_per_cluster))).squeeze().T)
            plt.title("Standard deviation\nper dimension")
            display(fig)
            plt.close(fig)
            
            
            fig = plt.figure(figsize=(8,4))
            set_inputs = train_inputs
            for i_class in range(len(set_inputs)):
                ins = var_to_np(set_inputs[i_class].squeeze())
                bps = np.abs(np.fft.rfft(ins.squeeze()))
                plt.plot(np.fft.rfftfreq(ins.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0))

                n_samples = 5000
                samples = get_samples(n_samples, i_class)
                inverted = var_to_np(invert_hierarchical(samples).squeeze())
                bps = np.abs(np.fft.rfft(inverted.squeeze()))
                plt.plot(np.fft.rfftfreq(inverted.squeeze().shape[1], d=1/ins.squeeze().shape[1]), np.median(bps, axis=0),
                        color=seaborn.color_palette()[i_class], ls='--')
            plt.title("Spectrum")
            plt.xlabel('Frequency [Hz]')

            plt.ylabel('Amplitude')
            plt.legend(['Real Right', 'Fake Right', 'Real Rest', 'Fake Rest'])
            display(fig)
            plt.close(fig)
            
            i_dims = (np.argsort(np.max(var_to_np(th.stack(stds_per_cluster)), axis=0))[::-1][:4])
            set_inputs = train_inputs
            for i_dim in i_dims:
                display_text("Dimension {:d}".format(i_dim))
                examples_per_class = []
                outs_per_class = []
                for i_class in range(2):
                    mean = means_per_cluster[i_class]
                    std = th.exp(stds_per_cluster[i_class])
                    i_f_vals = th.linspace((mean[i_dim] - 2 * std[i_dim]).item(),
                                           (mean[i_dim] +  2 *std[i_dim]).item(), 21)
                    examples = mean.repeat(len(i_f_vals), 1)
                    examples.data[:,i_dim] = i_f_vals.data
                    examples_per_class.append(examples.view(-1,33,2))
                    outs_per_class.append(feature_model(set_inputs[i_class]))
                #display_text(["Right", "Rest"][i_class])
                fig, axes = plt.subplots(1,2, figsize=(6,3), sharex=True, sharey=True)
                for i_class in range(len(train_inputs)):
                    from matplotlib import rcParams, cycler
                    cmap = plt.cm.coolwarm
                    N = len(examples)
                    examples = examples_per_class[i_class]
                    axes[i_class].plot(var_to_np(outs_per_class[i_class])[:,i_dim//2, i_dim%2].squeeze(),
                                       var_to_np(outs_per_class[i_class])[:,i_dim//2, i_dim%2].squeeze() * 0 - 0.01,
                                      ls='', marker='o', alpha=0.25, markersize=3,
                                      color=seaborn.color_palette()[i_class])
                    axes[i_class].scatter(var_to_np(examples)[:,i_dim//2, i_dim%2].squeeze(),
                                          var_to_np(examples)[:,i_dim//2, i_dim%2].squeeze() * 0,
                       c=cmap(np.linspace(0, 1, N)))
                    if i_class == 0:
                        axes[i_class].set_title("Latent space:")

                display(fig)
                plt.close(fig)
                with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
                    fig, axes = plt.subplots(1,2, figsize=(16,3), sharex=True, sharey=True)
                    for i_class in range(2):
                        inverted = invert_hierarchical(examples_per_class[i_class])
                        axes[i_class].plot(var_to_np(inverted).squeeze().T);
                    display(fig)
                    plt.close(fig)



In [ ]:
plt.figure(figsize=(4.5,2))
plt.plot(np.sin(np.linspace(0,8*np.pi, 64, endpoint=False)))
plt.title("Faster sine")

In [ ]:
plt.figure(figsize=(4.5,2))

plt.plot(np.sin(np.linspace(0,2*np.pi, 64, endpoint=False)))
plt.title("Slower sine")

In [ ]:
i_dims = (np.argsort(np.max(var_to_np(th.stack(stds_per_cluster)), axis=0)[::2])[::-1][:4]) * 2
set_inputs = train_inputs
for i_dim in i_dims:
    display_text("Dimension {:d}".format(i_dim))
    examples_per_class = []
    outs_per_class = []
    for i_class in range(2):
        mean = means_per_cluster[i_class]
        std = th.exp(stds_per_cluster[i_class])
        i_f_vals = th.linspace((mean[i_dim] - 2 * std[i_dim]).item(),
                               (mean[i_dim] +  2 *std[i_dim]).item(), 21)
        examples = mean.repeat(len(i_f_vals), 1)
        examples.data[:,i_dim] = i_f_vals.data
        examples_per_class.append(examples.view(-1,33,2))
        outs_per_class.append(feature_model(set_inputs[i_class]))
    #display_text(["Right", "Rest"][i_class])
    fig, axes = plt.subplots(1,2, figsize=(6,3), sharex=True, sharey=True)
    for i_class in range(len(train_inputs)):
        from matplotlib import rcParams, cycler
        cmap = plt.cm.coolwarm
        N = len(examples)
        examples = examples_per_class[i_class]
        axes[i_class].plot(var_to_np(outs_per_class[i_class])[:,i_dim//2, i_dim%2].squeeze(),
                           var_to_np(outs_per_class[i_class])[:,i_dim//2, i_dim%2].squeeze() * 0 - 0.01,
                          ls='', marker='o', alpha=0.25, markersize=3,
                          color=seaborn.color_palette()[i_class])
        axes[i_class].scatter(var_to_np(examples)[:,i_dim//2, i_dim%2].squeeze(),
                              var_to_np(examples)[:,i_dim//2, i_dim%2].squeeze() * 0,
           c=cmap(np.linspace(0, 1, N)))
        if i_class == 0:
            axes[i_class].set_title("Latent space:")

    display(fig)
    plt.close(fig)
    with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, N)))}):
        fig, axes = plt.subplots(1,2, figsize=(16,3), sharex=True, sharey=True)
        for i_class in range(2):
            inverted = invert_hierarchical(examples_per_class[i_class])
            axes[i_class].plot(var_to_np(inverted).squeeze().T);
        axes[0].set_title("Class A")
        axes[1].set_title("Class B")
        display(fig)
        plt.close(fig)



In [ ]:
mean_diff = means_per_cluster[1] - means_per_cluster[0]
mean_diff.data[1::2] = 0 # phase is same
plt.plot(var_to_np(mean_diff));

n_samples = 5
samples_A = get_samples(n_samples,0)

samples_B_amps = samples_A[:,:,0] + mean_diff[::2].unsqueeze(0)
samples_B = th.stack((samples_B_amps, samples_A[:,:,1]), dim=-1)

in_A = invert_hierarchical(samples_A)
in_B = invert_hierarchical(samples_B)

In [ ]:
fig, axes = plt.subplots(n_samples,2, figsize=(12,n_samples * 2), sharex=True, sharey=True);
for i_row in range(len(axes)):
    for i_col in range(len(axes[i_row])):
        ax = axes[i_row][i_col]
        if i_col == 0:
            ax.plot(var_to_np(in_A[i_row]).squeeze())
        else:
            ax.plot(var_to_np(in_B[i_row]).squeeze())
axes[0][0].set_title("Class A")
axes[0][1].set_title("Class B")


In [ ]:
plt.plot(var_to_np(th.stack(means_per_cluster)).T);

In [ ]:
plt.plot(var_to_np(th.exp(th.stack(stds_per_cluster))).T)

In [ ]:
stds_per_cluster

In [ ]:
i_class

In [ ]:
examples.shape